# Models

> Fill in a module description here


In [ ]:
#| default_exp FLearner

In [ ]:
#| hide
from nbdev.showdoc import *  # noqa: F403

## ToDo

- Add the TextBlock and The TabularBlock.
- Add loging using Loguru.

In [ ]:
#| export
import torch
import os
from datetime import datetime
from fastcore.utils import * # type: ignore # noqa: F403
from fedai.federated.agents import * # noqa: F403
from fedai.learner_utils import * # type: ignore # noqa: F403
from fedai.client_selector import *  # noqa: F403
from fedai.trainers import *  # noqa: F403
from fedai.core import get_cfg  # noqa: F401, F403
from fedai.wandb_writer import *  # noqa: F403

In [ ]:
#| export
class FLearner:
    def __init__(self,
                 cfg, # OmegaConf object
                 client_fn, # a function that returns a client object
                 client_selector= BaseClientSelector, # a client selection class represnting a client seleection algorithm # noqa: F405
                 client_cls= FLAgent,  # noqa: F405
                 trainer = Trainer,  # noqa: F405
                 loss_fn= torch.nn.CrossEntropyLoss,  # noqa: F405
                 writer= WandbWriter): # a writer to write results to an expirement tracking tool # noqa: F405
        
        self.cfg = cfg
        self.cfg.now = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.cfg.save_dir = os.path.join(self.cfg.save_dir, self.cfg.now)
        self.log_dir = os.path.join(self.cfg.log_dir, self.cfg.now)

        self.client_fn = client_fn
        self.client_selector = client_selector(self.cfg)
        self.client_cls = client_cls
        self.server  = self.client_cls(cfg= self.cfg, block= None, id= 0, state= None, role= AgentRole.SERVER)  # noqa: F405
        self.trainer = trainer
        self.loss_fn = loss_fn()
        self.writer = writer(cfg)

        self.latest_round = {}

    

In [ ]:
#| export
@patch
def run_simulation(self: FLearner):
    res =  []
    all_ids = self.client_selector.select()
    
    for t in range(1, self.cfg.n_rounds):
        lst_active_ids = all_ids[t]
        len_clients_ds = []
        round_res = []

        for id in lst_active_ids:
            client = self.client_fn(self.client_cls, self.cfg, id, self.latest_round, self.loss_fn)
            len_clients_ds.append(200) # FIX ME: this should be the length of the dataset of the client
            self.server.communicate(client, t) 

            trainer = self.trainer(client) 
            client_history = trainer.train() 
            round_res.append(client_history)
            res.append(round_res)

            client.communicate(self.server, t) 
            self.latest_round[id] = t 

        one_model = True if self.server.cfg.agg == 'one_model' else False
        self.server.aggregate(lst_active_ids, t, len_clients_ds, one_model= one_model) 
        
        if one_model:
            all_clients_ids = list(range(self.server.cfg.num_clients))
            for id in all_clients_ids:
                self.latest_round[id] = t

        self.writer.write(round_res, t) 
        
    self.writer.save(res)
    self.writer.finish()

    return res

In [ ]:
# client_selector = BaseClientSelector(cfg)  # noqa: F405
# client_cls = FLAgent # noqa: F405
# server = client_cls(cfg= cfg, block= None, id= 0, state= None, role= AgentRole.SERVER)
# latest_round = {}

In [ ]:
#| hide
import nbdev # type: ignore
nbdev.nbdev_export() # type: ignore  # noqa: E702
